In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

from catboost import Pool, cv,CatBoostClassifier

In [ ]:
train_df = pd.read_csv('train_df.csv')
test_df = pd.read_csv('test_df.csv')
submission_df = pd.read_csv('sample_submission.csv')

In [ ]:
cat_features = ['pickup_date', 'pickup_year', 'pickup_hour',
       'pickup_minute', 'drop_date', 'drop_year', 'drop_hour', 'drop_minute','pick_cluster']
labels = train_df['label'].values
train_df = train_df.drop(['label'], axis=1)

In [ ]:
params = {
    'iterations':300,
    'loss_function':'Logloss',
    'random_state':0,
#     'early_stopping_rounds':50,
    'cat_features':cat_features,
    'eval_metric':'F1'
}

In [ ]:
train_pool = Pool(data=train_df, label=labels,cat_features=cat_features)
test_pool = Pool(data=test_df, cat_features=cat_features)

In [ ]:
model = CatBoostClassifier(**params)

In [ ]:
model.fit(train_pool,verbose=10)

In [ ]:
model.get_feature_importance(prettified=True)

In [ ]:
for i, column in enumerate(train_df.columns):
    print(i, column)

In [ ]:
model.plot_tree(tree_idx=0)

In [ ]:
prediction = model.predict(test_pool)

In [ ]:
submission_df['prediction'] = prediction
submission_df.to_csv('submission.csv',index=False)